In [2]:
!conda install beautifulsoup4 -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         167 KB
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.1         |           py36_0         155 KB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    soupsieve-2.0              |             py_0          33 KB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     pkgs/main/linux-64::beautifulsoup4-4.9.0-py36_0
  soupsieve          pkgs/main/noarch::soupsieve-2.0-py_0

The following packages will be UPDATED:

  openssl            conda-fo

In [71]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
import requests

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_random_wikipedia_page=requests.get(wikipedia_link)
markup=raw_random_wikipedia_page.text
soupy = BeautifulSoup(markup, 'html.parser')
#print(soup.prettify())
soup=soupy.find_all('tbody')[0]
table=soup.find_all('th')
table                      

[<th>Postal Code
 </th>,
 <th>Borough
 </th>,
 <th>Neighborhood
 </th>]

In [8]:
#soup.find_all('tr')[1:]
poplu = []
bor = []
neigh = []

for data in soup.find_all('tr')[1:]:
    #soup.find_all('td')[0]
    poplu.append(data.find_all('td')[0].text)
    bor.append(data.find_all('td')[1].text)
    neigh.append(data.find_all('td')[2].text)
    

In [131]:
poplu
dfObj = pd.DataFrame({'PostalCode': poplu, 'Borough':bor, 'Neighborhood':neigh})
#dfObj['Neighborhood'] = dfObj.Neighborhood.str.replace('(\\n)', np.nan)
dfObj.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


In [132]:
dfObj = dfObj.replace('\n',' ', regex=True)

In [133]:
dfObj['Neighborhood'] = dfObj['Neighborhood'].replace(' ', np.nan)
dfObj[dfObj.columns] = dfObj.apply(lambda x: x.str.strip())

In [134]:
dfObj.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [135]:
dfObj = dfObj[dfObj.Borough != 'Not assigned']

In [136]:
dfObj.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

In [139]:
print(dfObj.shape)
print(dfObj.head())

(103, 3)
  PostalCode           Borough                                  Neighborhood
2        M3A        North York                                     Parkwoods
3        M4A        North York                              Victoria Village
4        M5A  Downtown Toronto                    Regent Park / Harbourfront
5        M6A        North York             Lawrence Manor / Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government
